In [135]:
from collections import defaultdict
import numpy as np
import gurobipy as grb
import pandas as pd

In [136]:
import pandas as pd

pmax = pd.read_csv('./pgmax.csv')
pmin = pd.read_csv('./pgmin.csv')
ru = pd.read_csv('./ramp.csv')
UT = pd.read_csv('./lu.csv')
DT = pd.read_csv('./ld.csv')    
demand = pd.read_csv('./Demand.csv')   
c_op = pd.read_csv('./cost_op.csv') 
c_st = pd.read_csv('./cost_st.csv') 
PTDF = pd.read_csv('./PTDF.csv') 
busgen = pd.read_csv('./busgen.csv')
busload = pd.read_csv('./busload.csv')
fmax = pd.read_csv('./fmax.csv')


PTDF = pd.DataFrame(PTDF)
busgen = pd.DataFrame(busgen)
busload = pd.DataFrame(busload)
demand = pd.DataFrame(demand)


# Convert the PTDF and demand string elements to a matrix
PTDF_matrix = np.vstack([list(map(float, row[0].split(';'))) for row in PTDF.values])
busgen_matrix = np.vstack([list(map(float, row[0].split(';'))) for row in busgen.values])
busload_matrix = np.vstack([list(map(float, row[0].split(';'))) for row in busload.values])
demand = np.vstack([list(map(float, row[0].split(';'))) for row in demand.values])

Hg = np.dot(PTDF_matrix, busgen_matrix)
Hl = np.dot(PTDF_matrix, busload_matrix)



In [137]:
model = grb.Model()

In [138]:
n_g=54
n_t=24
n_load=91
n_line=186
# b denotes on/off status and u denotes start start-up status
b = model.addVars(n_g, n_t, vtype=grb.GRB.BINARY)
u = model.addVars(n_g, n_t, vtype=grb.GRB.BINARY)
# how much to produce
p = model.addVars(n_g, n_t, vtype=grb.GRB.CONTINUOUS)
# slack variables
ls = model.addVars(n_t, 1, vtype=grb.GRB.CONTINUOUS)
ws = model.addVars(n_t, 1, vtype=grb.GRB.CONTINUOUS)
model.update()

In [139]:

for t in range(n_t):
    for j in range(n_g):
        model.addConstr(p[j,t] <= pmax.iloc[j].item() *b[j,t])
        model.addConstr(p[j,t] >= pmin.iloc[j].item() *b[j,t])
        if t >= 1:
            model.addConstr(p[j,t]- p[j,t-1] <= ru.iloc[j].item())
            model.addConstr(p[j,t-1]- p[j,t] <= ru.iloc[j].item())
            model.addConstr(u[j,t] >= b[j,t]-b[j,t-1])

In [140]:
for t in range(n_t):
    for j in range(n_g):
        if t >= 1:
            min_on_time = min(t + UT.iloc[j].item() - 1, n_t)
            for tau in range(t, min_on_time):
                model.addConstr(b[j, tau] >= b[j, t] - b[j, t - 1])

            min_off_time = min(t + DT.iloc[j].item() - 1, n_t)
            for tau in range(t, min_off_time):
                model.addConstr(1 - b[j, tau] >= b[j, t - 1] - b[j, t])


In [141]:
for t in range(n_t):
    model.addConstr(sum(p[j,t] for j in range(n_g)) == sum(demand[t, i] for i in range(n_load)))
model.update()

In [142]:
for t in range(n_t):
    for i in range(n_line):
        expr = sum(Hg[i, j] * p[j, t] for j in range(n_g)) - sum(Hl[i, j] * d_vector[j] for j in range(n_load))
        model.addConstr(expr <= fmax.iloc[i].item())

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [143]:
expr = sum(c_op.iloc[j].item() * p[j, t] for j in range(n_g) for t in range(n_t)) + sum(c_st.iloc[j].item() * u[j, t] for j in range(n_g) for t in range(n_t))
model.setObjective(sense=grb.GRB.MINIMIZE, expr=expr)
model.update()

In [144]:
opt = model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18494 rows, 3936 columns and 246002 nonzeros
Model fingerprint: 0x229f6d2c
Variable types: 1344 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [5e-07, 4e+02]
  Objective range  [8e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 4e+03]
Presolve removed 5797 rows and 78 columns
Presolve time: 0.15s
Presolved: 12697 rows, 3858 columns, 47626 nonzeros
Variable types: 1320 continuous, 2538 integer (2538 binary)

Root relaxation: objective 8.289218e+05, 1323 iterations, 0.08 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 828921.812    0   52

In [145]:
for t in range(1):
    for j in range(n_g):
        print(p.x)

AttributeError: 'tupledict' object has no attribute 'x'